In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


#  Packages & Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import sklearn as skl
import warnings
import statsmodels.api as sm

from sklearn import datasets, linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.model_selection import cross_val_score, cross_val_predict, ShuffleSplit, validation_curve, cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelBinarizer, RobustScaler, LabelEncoder, scale, MinMaxScaler, PolynomialFeatures
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_curve, roc_auc_score, confusion_matrix
from sklearn.decomposition import PCA
from sklearn import datasets
from sklearn.feature_selection import RFE,SelectFromModel

from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.datasets import make_classification

from xgboost import XGBClassifier 

from scipy.stats import skew, boxcox_normmax
from scipy.special import boxcox1p

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import math as m
import requests

warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter('ignore')


### Helper Functions

In [15]:
%load_ext autoreload
%aimport data_prep
%run data_prep.py
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load Datasets

In [10]:
train_data = pd.read_csv("training_set_values.csv")
train_labels = pd.read_csv("training_set_labels.csv")
test_data = pd.read_csv("test_set_values.csv")

### Columns cleaning

In [11]:
#Join Labels with Train data
train_data = addLabelToTrainData(train_data, train_labels)

#Drop id and recorded from train dataset 
train_data = prepareCols(train_data)
test_data = prepareCols(test_data)


`status_group` added to train_data 

['id', 'recorded_by', 'scheme_name', 'ward', 'wpt_name', 'subvillage', 'waterpoint_type_group'] removed from dataset 

['id', 'recorded_by', 'scheme_name', 'ward', 'wpt_name', 'subvillage', 'waterpoint_type_group'] removed from dataset 



### Shortlisting columns

Shortlist Trainning set columns

In [12]:
train_data_transformed = train_data.copy()


# installer
train_data_transformed = shortlist_installer(train_data_transformed)

# funder
train_data_transformed = shortlist_funder(train_data_transformed)

# lga
train_data_transformed = shortlist_lga(train_data_transformed)

# extraction_type
train_data_transformed = shortlist_extraction_type(train_data_transformed)

#scheme_management
train_data_transformed = shortlist_scheme_management(train_data_transformed)

#region_code
train_data_transformed = shortlist_region_code(train_data_transformed)




`installer` shortlisted to {'Commu', 'DANIDA', 'DWE', 'Government', 'RWE', 'other'} only 

`funder` shortlisted to {'Government Of Tanzania','Danida','Hesawa','Rwssp','World Bank','Kkkt','World Vision','Unicef','Tasaf','District Council', 'other'} only 

`lga` shortlisted to {'Njombe','Arusha Rural','Moshi Rural','Bariadi','Rungwe','Kilosa','Kasulu','Mbozi','Meru','Bagamoyo', 'other'} only 

`extraction_type` shortlisted to {'gravity','nira/tanira','submersible','swn 80','mono','india mark ii','afridev','ksb', 'other'} only 

`scheme_management` shortlisted to {'VWC','WUG','Water authority','WUA','Water Board','Parastatal','Private operator','Company', 'other'} only 

`region_code` shortlisted to {11,17,12,3,5,18,19,2,16,10,4,1,13,14,20, 'other'} only 



Shortlist Test set columns

In [13]:
test_data_transformed = test_data.copy()


# installer
test_data_transformed = shortlist_installer(test_data_transformed)

# funder
test_data_transformed = shortlist_funder(test_data_transformed)

# lga
test_data_transformed = shortlist_lga(test_data_transformed)

# extraction_type
test_data_transformed = shortlist_extraction_type(test_data_transformed)

#scheme_management
test_data_transformed = shortlist_scheme_management(test_data_transformed)

#region_code
test_data_transformed = shortlist_region_code(test_data_transformed)




`installer` shortlisted to {'Commu', 'DANIDA', 'DWE', 'Government', 'RWE', 'other'} only 

`funder` shortlisted to {'Government Of Tanzania','Danida','Hesawa','Rwssp','World Bank','Kkkt','World Vision','Unicef','Tasaf','District Council', 'other'} only 

`lga` shortlisted to {'Njombe','Arusha Rural','Moshi Rural','Bariadi','Rungwe','Kilosa','Kasulu','Mbozi','Meru','Bagamoyo', 'other'} only 

`extraction_type` shortlisted to {'gravity','nira/tanira','submersible','swn 80','mono','india mark ii','afridev','ksb', 'other'} only 

`scheme_management` shortlisted to {'VWC','WUG','Water authority','WUA','Water Board','Parastatal','Private operator','Company', 'other'} only 

`region_code` shortlisted to {11,17,12,3,5,18,19,2,16,10,4,1,13,14,20, 'other'} only 



### Impute Values 

In [16]:
# Impute 0 for regions of 'Dodoma','Kagera','Mbeya','Tabora' this regions 0 ar actually missing values
train_data_transformed = amount_tsh_impute_regions(train_data_transformed)

#Impute Latitude by the mean of the geographical areas (increasing order "subvillage", "ward", "lga", "district_code", "region", "basin")
train_data_transformed = impute_lat(train_data_transformed)

#Impute Longitude by the mean of the geographical areas (increasing order "subvillage", "ward", "lga", "district_code", "region", "basin")
train_data_transformed = impute_long(train_data_transformed)

#Impute Population by the mean of the geographical areas (increasing order "subvillage", "ward", "lga", "district_code", "region", "basin")
train_data_transformed = impute_pop(train_data_transformed)


NameError: name 'm' is not defined

In [ ]:
# Impute 0 for regions of 'Dodoma','Kagera','Mbeya','Tabora' this regions 0 ar actually missing values
test_data_transformed = amount_tsh_impute_regions(test_data_transformed)

#Impute Latitude by the mean of the geographical areas (increasing order "subvillage", "ward", "lga", "district_code", "region", "basin")
test_data_transformed = impute_lat(test_data_transformed)

#Impute Longitude by the mean of the geographical areas (increasing order "subvillage", "ward", "lga", "district_code", "region", "basin")
test_data_transformed = impute_long(test_data_transformed)

#Impute Population by the mean of the geographical areas (increasing order "subvillage", "ward", "lga", "district_code", "region", "basin")
test_data_transformed = impute_pop(test_data_transformed)


### Add Outside Data

In [ ]:
train_data = density(train_data)
test_data = density(test_data)

### Conversions
#### Convert `construction_year`

In [ ]:
# construction_year - converts it to years elapsed (AKA age) -- (zeroes ignored)
train_data_transformed = convert_construction_year(train_data_transformed)

In [ ]:
# construction_year - converts it to years elapsed (AKA age) -- (zeroes ignored)
test_data_transformed = convert_construction_year(test_data_transformed)

### ^^^^^^^^^^^^^^^^^^^^^^^^^^ What to do with zeros here ? (summary  ↓ ) -- currently imputing with median

In [ ]:
zeroes =    train_data_transformed['age'][train_data_transformed['age'] == 0].count() / train_data_transformed['age'].count()
mean = np.mean(train_data_transformed['age'][train_data_transformed['age'] != 0])
median = np.median(train_data_transformed['age'][train_data_transformed['age'] != 0])

print("% of zeroes: {}".format(zeroes))

print("mean: {}".format(mean))


print("median: {}".format(median))

# impute with median
train_data_transformed[train_data_transformed['age'] == 0][["age"]] = median

In [ ]:
#Apply same process for Test Dataset

#### Convert `date_recorded`

In [ ]:
train_data_transformed = convert_date_recorded(train_data_transformed)

In [ ]:
test_data_transformed = convert_date_recorded(test_data_transformed)

In [ ]:
### Mapping long and lat to city

### Binning

In [ ]:
train_data_transformed = bin_feature(train_data_transformed, "days_since_recoreded", 10)

In [ ]:
test_data_transformed = bin_feature(test_data_transformed, "days_since_recoreded", 10)

In [ ]:
numerical = ['amount_tsh', 'gps_height', 'num_private', 'population']
other = ['longitude', 'latitude', "status_code"]
categorical = list(set(train_data_transformed.columns) - set(numerical) - set(other))
categorical

### Fixing the Skewness

In [ ]:
train_temp['latitude'] = train_temp["latitude"].astype('category') 
train_temp['longitude'] = train_temp["longitude"].astype('category') 

test_temp['latitude'] = test_temp["latitude"].astype('category') 
test_temp['longitude'] = test_temp["longitude"].astype('category') 

In [ ]:
feature_skewness(test_temp)
feature_skewness(train_temp)


In [ ]:
test_temp = fix_skewness(test_temp)
train_temp = fix_skewness(train_temp)

# Correlation matrix

In [ ]:
plt.pyplot.figure(figsize=(20,10))
sns.heatmap(train_data_transformed.corr(),cbar=True,fmt =' .2f', annot=True, cmap='viridis')

# Hot Encode & Save Data

In [ ]:
len(train_temp.columns)

In [ ]:
len(test_temp.columns)

In [14]:
#train_data_transformed.status_group.replace(['functional', 'non functional','functional needs repair'], [1, 2, 3], inplace=True)
labels_prepared = train_data_transformed[["status_group"]]
train_data_transformed = train_data_transformed.drop(columns=["status_group"])
#train_data_transformed = onehot_encode(train_data_transformed)
train_data_transformed.to_csv("train_data_prepared.csv")
labels_prepared.to_csv("labels_prepared.csv")

In [ ]:
test_data_transformed.to_csv("test_data_prepared.csv")